> This notebook contains the evolutionary dataset first analysis.

In [3]:
import pandas as pd
import numpy as np

import re
import csv

from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os

### 1. Data Loading

> We have both xls and fasta format for the evolutionary dataset. 

In [4]:
# Data Paths
FASTA_msa_seq = 'data/msa/4.Fasta_all_7k.txt'
XLS_msa_seq = 'data/msa/Dataset_S4.xlsx'
XLS_msa_seq_csv = 'data/msa/Dataset_S4.csv'

# Read the Excel file and write to CSV
df = pd.read_excel(XLS_msa_seq)
df.to_csv(XLS_msa_seq_csv, index=False)

# Data Loading
#fasta_msa = pd.read_csv(FASTA_msa_seq,  delimiter='\t')
XLS_msa = pd.read_csv(XLS_msa_seq_csv)



c:\Users\kleka\anaconda3\envs\ml\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [5]:
display(XLS_msa)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,NaN,User note: GenBank Accession numbers marked wi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Database Source,Sequence ID,GenBank Accession,Primary Structure,GXNCRFLQ Motif,Protein Length,Domain Structure,Functional Cluster,Primary Effector,Primary Effector Gene Ontology,Linker Length,Number of Predicted Transmembrane Helices,Transmembrane Helix Topology,Kingdom,Phylum,Class,Order,Family,Genus,Species
3,Interpro,A0A009EVU7,EXA65901.1,MTSFYNGGTSQEIFNLIGQAAALEDIIDAITVWLGDKIPDALVSVM...,GKNCRILQ,850.0,"[N_terminus : (0, 0) ---> GAF_2 : (9, 166) ---...","['EAL', 'GAF', 'GGDEF', 'LOV', 'PAS']",GGDEF,Cyclic Nucleotide Biosythesis,12.0,0.0,NaN,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,baumannii
4,Interpro,A0A009JQP1,EXB31618.1,MNNNCDDHDTYHIFNLIGQHTELHLILSAASTWLESRISNVLVAIM...,GKTCHFLQ,855.0,"[N_terminus : (0, 0) ---> GAF_2 : (26, 166) --...","['EAL', 'GAF', 'GGDEF', 'LOV', 'PAS']",GGDEF,Cyclic Nucleotide Biosythesis,12.0,0.0,NaN,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,baumannii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6780,OneKP,ZYAX_2075737,KU702188,MEPSNPGNTNNSFPSGNVVRKVLDDSPAIISPLPRDSRGSLEVFNP...,GKNCRFLQ,1083.0,"[N_terminus : (0, 0) ---> TLOV : (286, 663) --...","['Pkinase', 'TLOV']",Pkinase,Kinase Activity,83.0,0.0,NaN,Land Plants,Pinophyta,Pinopsida,Pinales,Taxaceae,Taxus,cuspidata
6781,OneKP,ZZOL_2003873,KU702189,MESTSGSKQDHYFSQRSSSMPKPVRDSLAQLPRDSRGSLEVFNPQG...,GRNCRFLQ,1008.0,"[N_terminus : (0, 0) ---> TLOV : (232, 597) --...","['Pkinase', 'TLOV']",Pkinase,Kinase Activity,83.0,0.0,NaN,Land Plants,Lycopodiophyta,Isoetopsida,Selaginellales,Selaginellaceae,Selaginella,lepidophylla
6782,OneKP,ZZOL_2007563,KU702190,MALQNGSLLSSDRKCVRSPRMKNHKLIVSQSLIRSYRASIQDELQK...,GRNCRFLQ,394.0,"[N_terminus : (0, 0) ---> TLOV : (49, 354) ---...",['TLOV'],Short_LOV,Short LOV,0.0,0.0,NaN,Land Plants,Lycopodiophyta,Isoetopsida,Selaginellales,Selaginellaceae,Selaginella,lepidophylla
6783,OneKP,ZZOL_2011001,KU702191#,MLVEVSQFTEGSKDKAVRPNGMPESLIKYDARQKDSVVSSVSELVE...,GKNCRFLQ,465.0,"[N_terminus : (0, 0) ---> LOV : (142, 247) ---...","['LOV', 'Pkinase']",Pkinase,Kinase Activity,83.0,0.0,NaN,Land Plants,Lycopodiophyta,Isoetopsida,Selaginellales,Selaginellaceae,Selaginella,lepidophylla


###  2. Generate the sequences for MSA

In [6]:

msa_align = 'data/msa/msa_align.txt'
df = pd.read_csv(XLS_msa_seq_csv)

# # Find the fourth column element of the row where the second column has the value 'Q2NB89'
# target_value = df[df.iloc[:, 1] == 'Q2NB98'].iloc[0, 3]

# # Extract the fourth column of all rows starting from index 3
# selected_column = df.iloc[3:, 3]

# # Prepend the target value to the selected column data
# result_series = pd.concat([pd.Series([target_value]), selected_column])

# # Write the result to the text file
# result_series.to_csv(msa_align, index=False, header=False)

# Initialize a list to store formatted strings
formatted_rows = []

# Check if there's a row where the second column has the value 'Q2NB89' and add it first
if 'Q2NB98' in df.iloc[:, 1].values:
    target_row = df[df.iloc[:, 1] == 'Q2NB98'].iloc[0]
    formatted_rows.append('>' + str(target_row.iloc[1]) + '\n' + str(target_row.iloc[3]))

# Format and add the remaining rows starting from index 3
for _, row in df.iloc[3:].iterrows():
    formatted_rows.append('>' + str(row.iloc[1]) + '\n' + str(row.iloc[3]))

# Join the formatted rows with a newline character
formatted_string = '\n'.join(formatted_rows)

# Write the result to the text file
with open(msa_align, 'w', encoding='utf-8') as file:
    file.write(formatted_string)

>Here we generate the two files that will be used for msa. We are only interested in the LOV domain of the WT protein (Q2NB98) - from the 16th amino acid (GADD...) to the 168th (...KTLS).

In [7]:
# File paths
WT_path = 'data/msa/WT_align.txt'  # File for the first two lines
sequences_path = 'data/msa/msa_comparisons.txt'  # File for the rest of the lines

# Read the original file
with open(msa_align, 'r', encoding='utf-8') as file:
    lines = file.readlines()


if len(lines) > 1:
    lines[1] = lines[1] + '\n'

# Write the first two lines to the first file
with open(WT_path, 'w', encoding='utf-8') as file:
    file.writelines(lines[:2])

# Write the rest of the lines to the second file
with open(sequences_path, 'w', encoding='utf-8') as file:
    file.writelines(lines[2:])

###  2. Convert MSA fasta file to adequate .a2m format

> Look at EVMutation git (2. Computing pairwise models of sequences) the alignment requirements are the folowing:
The first step of the EVmutation method is to compute a pairwise model of sequences from a family sequence alignment, e.g. the included example file (example/msa_comparison.a2m). This alignment has to be in aligned FASTA/A2M format and must fulfill the following requirements:
* The target sequence may not contain any gaps.
* Columns that should be excluded from model inference (e.g. too many gaps) must be represented by lowercase characters. Gaps in these positions must be represented by a dot (".") rather than a dash ("-").
* The identifier of the target sequence has to be passed to plmc with the -f parameter (see below)

In [8]:
def process_alignment(fasta_file, start, stop, dots_to_dashes=True):
    """
    Processes a protein multiple sequence alignment.

    Args:
    fasta_file (str): Path to the input FASTA file.
    start (int): Start index of the region of interest (1-based, inclusive).
    stop (int): Stop index of the region of interest (1-based, inclusive).
    dots_to_dashes (bool): If True, convert all dashes to dots in the sequences.
                           If False, remove positions with dashes in the WT sequence.

    Raises:
    ValueError: If start or stop indices are out of bounds.
    FileNotFoundError: If the input file is not found.
    """

    try:
        # Read the alignment from the FASTA file
        alignment = AlignIO.read(fasta_file, "fasta")
    except FileNotFoundError:
        raise FileNotFoundError(f"File {fasta_file} not found.")

    # Get the WT sequence (first sequence in the alignment)
    wt_sequence = str(alignment[0].seq)

    # Check the dot-to-dash conversion or dash removal requirement
    if dots_to_dashes:
        wt_sequence = wt_sequence.replace('-', '.')
        # Convert dashes to dots in all sequences
        for record in alignment:
            record.seq = Seq(str(record.seq).replace('-', '.'))
    else:
        # Remove positions where WT sequence has a dash
        non_dash_positions = [i for i, char in enumerate(wt_sequence) if char != '-']
        new_alignment = MultipleSeqAlignment([])
        for record in alignment:
            new_seq = ''.join(record.seq[i] for i in non_dash_positions)
            new_alignment.append(SeqRecord(Seq(new_seq), id=record.id, description=record.description))
        alignment = new_alignment
        wt_sequence = ''.join(char for char in wt_sequence if char != '-')

    # Find amino acid positions to identify start and stop
    aa_positions = [i for i, char in enumerate(wt_sequence) if char != '.']

    # Check if start and stop are within the bounds of the WT sequence
    if start < 1 or stop > len(aa_positions):
        raise ValueError("Start and stop indices are out of the sequence bounds.")

    # Convert start and stop from 1-based to 0-based and from amino acid count to alignment position
    start_index = aa_positions[start - 1]
    stop_index = aa_positions[stop - 1] if stop <= len(aa_positions) else len(wt_sequence) - 1

    # Process each sequence in the alignment
    processed_alignment = MultipleSeqAlignment([])
    for record in alignment:
        sequence = str(record.seq)
        processed_seq = (
            sequence[:start_index].lower() +
            sequence[start_index:stop_index + 1] +
            sequence[stop_index + 1:].lower()
        )
        processed_record = SeqRecord(Seq(processed_seq), id=record.id, description=record.description)
        processed_alignment.append(processed_record)

    # Save the processed alignment to an .a2m file
    output_file = fasta_file.rsplit('.', 1)[0] + ".a2m"
    AlignIO.write(processed_alignment, output_file, "fasta")



In [9]:
def convert_dashes_to_dots(fasta_file):
    """
    Processes a protein multiple sequence alignment in .a2m format. Wherever there are lowercase
    letters in the WT sequence and dashes in the corresponding positions in the other
    sequences, converts the dashes to dots. If there is a lowercase letter in the
    corresponding position of the other sequences, it is left as is.

    Args:
    fasta_file (str): Path to the input .a2m file.

    Raises:
    FileNotFoundError: If the input file is not found.
    """

    try:
        # Read the alignment from the .a2m file
        alignment = AlignIO.read(fasta_file, "fasta")
    except FileNotFoundError:
        raise FileNotFoundError(f"File {fasta_file} not found.")

    # Get the WT sequence (first sequence in the alignment)
    wt_sequence = str(alignment[0].seq)

    # Process each sequence in the alignment
    processed_alignment = MultipleSeqAlignment([])
    for record in alignment:
        sequence = str(record.seq)
        processed_seq = ""
        for i, char in enumerate(sequence):
            wt_char = wt_sequence[i]
            if wt_char.islower() and char == '-':
                processed_seq += '.'
            else:
                processed_seq += char
        processed_record = SeqRecord(Seq(processed_seq), id=record.id, description=record.description)
        processed_alignment.append(processed_record)

    # Save the processed alignment to a new .a2m file
    output_file = "Q2NB98.a2m"
    AlignIO.write(processed_alignment, output_file, "fasta")

>Let's apply this function to the fasta file (LOV domains starting with EL222 aligned.fasta).

In [10]:
LOV_msa_fasta = 'EVmutation\example\LOV_domains_starting_with_EL222_aligned.fasta'
start_index = 16
stop_index = 168
process_alignment(LOV_msa_fasta, start_index, stop_index, dots_to_dashes=False)


In [11]:
LOV_msa_a2m_dashes = 'EVmutation\example\LOV_domains_starting_with_EL222_aligned.a2m'
convert_dashes_to_dots(LOV_msa_a2m_dashes)

>Function to remove sequences that have more than 30% gaps.

In [12]:
def remove_high_gap_sequences(input_file, output_file, gap_threshold=30):
    """
    Removes sequences from an alignment file (.a2m) that have more than a certain
    percentage of gaps.

    Args:
    input_file (str): Path to the input .a2m file.
    output_file (str): Path to save the filtered .a2m file.
    gap_threshold (int): The threshold percentage of gaps (default 30%).
    """

     # Read the alignment
    alignment = AlignIO.read(input_file, "fasta")
    total_sequences = len(alignment)

    # Filter sequences with gap percentage less than the threshold
    filtered_sequences = [
        record for record in alignment if ((str(record.seq).count('-') + str(record.seq).count('.')) / len(record.seq)) * 100 < gap_threshold
    ]


    # Rename the first sequence
    filtered_sequences[0].id = 'Q2NB98_30perc'
    filtered_sequences[0].name = 'Q2NB98_30perc'
    filtered_sequences[0].description = ''  # Clearing the description


    # Create a new alignment with the filtered and renamed sequences
    filtered_alignment = MultipleSeqAlignment(filtered_sequences)

    # Write the filtered alignment to a file
    AlignIO.write(filtered_alignment, output_file, "fasta")

    # Calculate the number and percentage of removed sequences
    num_removed = total_sequences - len(filtered_sequences)
    perc_removed = (num_removed / total_sequences) * 100

    return num_removed, perc_removed

In [13]:
LOV_msa_a2m_30gaps_removed = 'Q2NB98_30perc.a2m'
num_removed, perc_removed = remove_high_gap_sequences(LOV_msa_a2m_dashes, LOV_msa_a2m_30gaps_removed, gap_threshold=30)
print(f"Number of sequences removed: {num_removed}")
print(f"Percentage of sequences removed: {perc_removed:.2f}%")

Number of sequences removed: 4000
Percentage of sequences removed: 58.98%


###  3. Find the value for the L2 regularizer (to run PLMC)

>We adjust the strength of $l_{2}$-regularization $\lambda_J$ on the couplings based on the length of the model that we infer as $0.2 \cdot (N - 1)$, where $N$ is the length of the model. $N$ corresponds to the number of uppercase positions in the alignment file. For the included LOV_domains example, there are 151 uppercase residues in the LOV sequence, which means that we choose a regularization strength of $\lambda_J = 0.2 \cdot (151 - 1) = 30$. This value is passed to plmc using the -le command line option. To run PLMC we use the following command:
plmc/bin/plmc -o example/LOV_domains_starting_with_EL222_aligned.model_params -c example/LOV_domains_starting_with_EL222_aligned.txt -f Q2NB98 -le 30 -lh 0.01 -m 200 -t 0.2 -g example/LOV_domains_starting_with_EL222_aligned.a2m

###  4. Function that adds start and stop for domain of interest

In [20]:
def extract_lov_domain_tuples(df):
    lov_domain_data = []

    for index, row in df.iterrows():
        # Skip initial rows that don't contain the relevant data
        if index < 3:  # Assuming data starts from the fifth row (index 3)
            continue

        # Access columns by position using .iloc
        protein_id = row.iloc[1]  # Assuming the protein ID is in the second column
        domain_info = row.iloc[6]  # Assuming the domain info is in the sixth column

        print(f"Debug: Protein ID: {protein_id}, Domain Info: {domain_info}")

        # Regular expression to find the LOV domain tuple
        match = re.search(r'LOV : \((\d+), (\d+)\)', domain_info)
        if match:
            start, end = match.groups()
            lov_domain_data.append((protein_id, (int(start), int(end))))

    return lov_domain_data

In [21]:
lov_data = extract_lov_domain_tuples(XLS_msa)
# print(lov_data)

Debug: Protein ID: A0A009EVU7, Domain Info: [N_terminus : (0, 0) ---> GAF_2 : (9, 166) ---> PAS_4 : (186, 293) ---> LOV : (309, 414) ---> GGDEF : (426, 583) ---> EAL : (601, 839) ---> C_terminus : (850, 850)]
Debug: Protein ID: A0A009JQP1, Domain Info: [N_terminus : (0, 0) ---> GAF_2 : (26, 166) ---> PAS_4 : (187, 293) ---> LOV : (309, 414) ---> GGDEF : (426, 583) ---> EAL : (604, 840) ---> C_terminus : (855, 855)]
Debug: Protein ID: A0A009NCY2, Domain Info: [N_terminus : (0, 0) ---> GAF_2 : (26, 166) ---> PAS_4 : (187, 293) ---> LOV : (309, 414) ---> GGDEF : (426, 583) ---> EAL : (604, 840) ---> C_terminus : (855, 855)]
Debug: Protein ID: A0A010FRR9, Domain Info: [N_terminus : (0, 0) ---> GAF_2 : (26, 166) ---> PAS_4 : (187, 293) ---> LOV : (309, 414) ---> GGDEF : (426, 583) ---> EAL : (604, 840) ---> C_terminus : (855, 855)]
Debug: Protein ID: A0A010PA15, Domain Info: [N_terminus : (0, 0) ---> LOV : (450, 555) ---> HisKA : (576, 641) ---> HATPase_c : (688, 803) ---> Response_reg : (8

In [22]:
def update_a2m_with_domain_ranges(a2m_file, updated_a2m_file, lov_domain_data):
    """
    Updates sequence IDs in an .a2m file with domain ranges.

    Args:
    a2m_file (str): Path to the input .a2m file.
    updated_a2m_file (str): Path for the output .a2m file with updated IDs.
    lov_domain_data (list): List of tuples containing (protein_id, (start, end)).
    """
    # Load the alignment from the .a2m file
    alignment = AlignIO.read(a2m_file, "fasta")

    # Create a dictionary from the lov_domain_data for easy lookup
    domain_dict = dict(lov_domain_data)

    # Update sequence IDs with domain ranges
    for record in alignment:
        protein_id = record.id
        if protein_id in domain_dict:
            start, end = domain_dict[protein_id]
            record.id = f"{protein_id}/{start}-{end}"
            record.description = ''  # Clearing the description to avoid duplication

    # Write the updated alignment to a new .a2m file
    AlignIO.write(alignment, updated_a2m_file, "fasta")

In [23]:
update_a2m_with_domain_ranges('Q2NB98.a2m', 'Q2NB98_range.a2m', lov_data)